In [ ]:
!pip install natasha
!pip install yargy
!pip install ipymarkup
!pip install graphviz

In [4]:
from yargy import rule, Parser, or_, and_, not_
from yargy.predicates import eq, type,gram, is_capitalized
from ipymarkup import show_span_ascii_markup as show_markup
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy.predicates import gram
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline


#Data

##Names

In [401]:
import pandas as pd
import numpy as np
Path = "/content/drive/MyDrive/NLP2/"
names = pd.read_csv(Path + 'russian_names.csv')
surn =  pd.read_csv(Path + 'russian_surnames.csv')
names_list = []
surnames_list = []
for name in names.Name:
  names_list.append(name)
for sname in surn.Surname:
  surnames_list.append(sname)  
surnames_list[:10]

['Аалферов',
 'Аалферова',
 'Ааль',
 'Ааман',
 'Аамана',
 'Ааманая',
 'Ааманий',
 'Аандреев',
 'Аандреева',
 'Аарон']

#Code

##Name extractor

In [418]:

def PersonNERHypothesis(text):
  text = text.title().split()
  new = ''
  mc = 0
  nc = 0
  sc = 0
  namesl = []
  surnl= []
  for word in text:
    for name in names_list:
      if name in word and (word.endswith('вна') or word.endswith('ич')) and mc == 0:
        new += word + ' '
        # print('add middle')
        mc = 1
      if name == word and nc == 0:
        namesl = [name]
        new += name + ' '
        # print('add name')
        nc = 1
      if nc == 1 and mc == 1:
        break

  for word in text:
    for name in surnames_list:
      if name == word:
        surnl = [name]
        new += name + ' '
        # print(surnl)
        sc = 1
        break
    if sc ==1:
      break   
      

  Full_name = fact(
      'Name',
      ['name','surn','middle']
  )
  ABBR = gram('Abbr')
  TITLE = is_capitalized()
  PATR = and_(
      gram('Patr'),
      not_(ABBR)
  )

  MIDDLE = PATR.interpretation(
      Full_name.middle
  )

  MIDDLE_ABBR = and_(
      ABBR,
      TITLE
  ).interpretation(
      Full_name.middle
  )

  NAME = morph_pipeline(namesl).interpretation(
      Full_name.name.normalized()
  )

  SURN = morph_pipeline(surnl).interpretation(
      Full_name.surn.normalized()
  )

  gnc = gnc_relation()
  FULL_NAME = or_(
      rule(
        NAME.interpretation(Full_name.name).optional(),
        SURN.interpretation(Full_name.surn).optional(),
        MIDDLE.interpretation(Full_name.middle).optional(),
  ),rule(
        SURN.interpretation(Full_name.surn).optional(),
        MIDDLE.interpretation(Full_name.middle).optional(),
        NAME.interpretation(Full_name.name).optional()
  ),rule(
        NAME.interpretation(Full_name.name).optional(),
        MIDDLE.interpretation(Full_name.middle).optional(),
        SURN.interpretation(Full_name.surn).optional()
  )
  ).interpretation(
      Full_name
  )

  parser = Parser(FULL_NAME)


  # print(new)
  try:
    match = parser.match(new)
    match.fact
    first = match.fact.name
    middle = match.fact.middle
    last = match.fact.surn
    # print((match.fact))
    
  except ValueError:  
    first = None
    middle = None
    last = None

  return first, middle, last

##Test case

In [419]:
#Не разобрался как использовать unittests, делаю так :)
testing_address = ['Иванов Петр Васильевич','шипицын дмитрий вячеславович','елена владимировна','басалаева юлия михайловна','ну я как раз по фамилии есть смотри мам',
                   'глушенков власти на android','фамилию сказать что за фамилия терентьева людмила','анюта','р1 артем витальевич','фитнес веретельников олег викторович']
valid = [('Петр','Васильевич','Иванов'),('Дмитрий','Вячеславович','Шипицын'),('Елена','Владимировна',None),('Юлия','Михайловна','Басалаева'),(None, None, None), (None,None, 'Глушенков'),
         ('Людмила', None, 'Терентьева'), ('Анюта',None,None), ('Артем','Витальевич',None), ('Олег','Викторович','Веретельников')]
for test in range(len(valid)):
  personNER = PersonNERHypothesis(testing_address[test])
  print('Test: ', test ,' ', personNER == valid[test])



Test:  0   True
Test:  1   True
Test:  2   True
Test:  3   True
Test:  4   True
Test:  5   True
Test:  6   True
Test:  7   True
Test:  8   True
Test:  9   True
